# установка + настройка датасета

Роман Бохян romanbokhyan@yandex.ru обучал колаб базовый

заинсталлю все необходимые библиотеки

In [1]:
%pip install -U wordfreq transformers regex datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: regex
    Found existing installation: regex 2024.11.6
    Uninstalling regex-2024.11.6:
      Successfully uninstalled regex-2024.11.6
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Fou

In [2]:
import pandas as pd
import numpy as np
import torch
import math
import gzip
import unicodedata
import random
import re
import regex
import datasets
from datasets import Dataset, DatasetDict
from wordfreq import zipf_frequency, top_n_list
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "cuda" if torch.cuda.is_available() else "cpu"
)
device

device(type='cpu')

In [5]:
device_cpu = torch.device('cpu')
device_cpu

device(type='cpu')

начну работу с датасетом. посмотрим просто что там

In [7]:
task_data = pd.read_csv(
    "/content/dataset_1937770_3 (1).txt",
    sep="\t",
)
task_data

,"id,text_no_spaces"
0,"0,куплюайфон14про"
1,"1,ищудомвПодмосковье"
2,"2,сдаюквартирусмебельюитехникой"
3,"3,новыйдивандоставканедорого"
4,"4,отдамдаромкошку"
...,...
1000,"1000,Янеусну."
1001,"1001,Весна-яуженегреюпио."
1002,"1002,Весна-скоровырастеттрава."
1003,"1003,Весна-выпосмотрите,каккрасиво."


In [8]:
task_data.columns

Index(['id,text_no_spaces'], dtype='object')

так ну сразу видно что айди наслаивается на text_no_spaces. я разделю эти столбцы чтобы потом проще было

In [9]:
s = task_data['id,text_no_spaces'].astype(str)
task_data[['id', 'text_no_spaces']] = s.str.split(pat=',', n=1, expand=True)
task_data['id'] = pd.to_numeric(task_data['id'], errors='raise')
task_data['text_no_spaces'] = task_data['text_no_spaces'].fillna('').str.strip()
task_data = task_data.drop(columns=['id,text_no_spaces'])

In [10]:
task_data


,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
...,...,...
1000,1000,Янеусну.
1001,1001,Весна-яуженегреюпио.
1002,1002,Весна-скоровырастеттрава.
1003,1003,"Весна-выпосмотрите,каккрасиво."


еще важно сделать мини еда анализ датасета, мало ли где то что то криво


In [11]:
na_counts = task_data.isna().sum().sort_values(ascending=False)
na_counts

,0
id,0
text_no_spaces,0


In [12]:
has_spaces = task_data['text_no_spaces'].str.contains(r'\s', regex=True)
if has_spaces.any():
    display(task_data.loc[has_spaces, ['id','text_no_spaces']].head(10))

In [13]:
def char_bucket_counts(s,other_to_check,latin_to_check):
    buckets = {'cyrillic':0,'latin':0,'digits':0,'hyphen':0,'punct':0,'other':0}
    for ch in s:
        if 'а' <= ch <= 'я' or 'А' <= ch <= 'Я' or ch in 'ёЁ':
            buckets['cyrillic'] += 1
        elif 'a' <= ch <= 'z' or 'A' <= ch <= 'Z':
            buckets['latin'] += 1
            latin_to_check.append(ch)
        elif ch.isdigit():
            buckets['digits'] += 1
        elif ch == '-':
            buckets['hyphen'] += 1
        elif ch in '.,;:!?%)»]}…':
            buckets['punct'] += 1
        else:
            buckets['other'] += 1
            other_to_check.append(ch)
    return buckets
other_to_check = []
latin_to_check = []
sample = ''.join(task_data['text_no_spaces'])
b = char_bucket_counts(sample, other_to_check, latin_to_check)
total = sum(b.values()) or 1
print(b)
print(other_to_check)
print(set(latin_to_check))

{'cyrillic': 21693, 'latin': 498, 'digits': 50, 'hyphen': 62, 'punct': 641, 'other': 31}
['’', '—', '—', '—', '–', '–', '–', '–', '+', "'", "'", "'", "'", "'", "'", "'", "'", '–', '«', '«', '«', '«', '–', '«', '«', "'", "'", "'", "'", "'", "'"]
{'M', 'p', 't', 'S', 'i', 'w', 'n', 'L', 'k', 'd', 'R', 'U', 'm', 'O', 'l', 'r', 'V', 'P', 'a', 'y', 'G', 'E', 'c', 'x', 'u', 'B', 'F', 'e', 'D', 'X', 'h', 'v', 'K', 'H', 'J', 'T', 'b', 'I', 'A', 'Y', 's', 'C', 'N', 'W', 'g', 'o'}


In [14]:
latin_word_pattern = re.compile(r'\b\w*[a-zA-Z]\w*\b')

def contains_latin_word(text):
    return bool(latin_word_pattern.search(text))

rows_with_latin = task_data[task_data['text_no_spaces'].apply(contains_latin_word)]
display(rows_with_latin)

,id,text_no_spaces
6,6,куплютелевизорPhilips
9,9,куплюноутбукHP
11,11,новаямикроволновкаSamsung
13,13,куплюгитаруFender
19,19,куплюXboxOne
...,...,...
419,419,ноутбукAsusROG
632,632,Mitornodiaboloesta.
765,765,"ВгородеNдожди,"
774,774,"ВгородеNзима,"


в целом мини еда (я проверил базу которая могла бы подвести меня по типу пробелов, пропусков и какое кол-во каких чаров в датасете) показала что все норм и можно приступать дальше. как видим тут в основном кириллица а англ слова либо рандомные либо название компании (ну либо ссср) )

# Эксперименты. count based approach.
если что я каждый раз перезапускал ядро перед каждым новым экспериментом. если вдруг кто то будет перезапускать мой код - учтите, что надо реранить все ячейки перед каждым экспом (все что до этого и перед # посылка, в зависимости от посылки надо запускать ячейки) чтобы резы были как у меня

In [ ]:
df = task_data

In [ ]:
def norm(s):
    return unicodedata.normalize('NFKC', s).lower()

def build_dict(n_ru=200_000, n_en=150_000):
    ru = top_n_list('ru', n=n_ru, wordlist='best')
    en = top_n_list('en', n=n_en, wordlist='best')
    vocab = set(ru) | set(en)

    cost = {}
    mx = 0
    for w in vocab:
        z_ru = zipf_frequency(w, 'ru')
        z_en = zipf_frequency(w, 'en')
        z = max(z_ru, z_en)
        c = -(z) + 0.05 * max(0, 8 - len(w))
        cost[norm(w)] = c
        if len(w) > mx:
            mx = len(w)
    mx = min(max(mx, 24), 32)
    return cost, mx


In [ ]:
WORD_COST, _MAX = build_dict(n_ru=600_000, n_en=400_000)
MAX_WORD = 32

RU = r'\p{IsCyrillic}'
EN = r'\p{IsLatin}'
DG = r'\p{Nd}'

re_boundary = re.compile(
    rf'((?<=[{RU}])(?=[{EN}{DG}]))|'   # ру -> лат/цифры
    rf'((?<=[{EN}])(?=[{RU}{DG}]))|'   # лат -> ру/цифры
    rf'((?<=[{DG}])(?=[{RU}{EN}]))|'   # цифры -> буквы
    rf'(?<=[\p{{Ll}}])(?=\p{{Lu}})'    # aБ / aA
)

PUNCT_RIGHT = set('.,;:!?%)»]}…')
PUNCT_LEFT = set('([«[{')

BASE_PER_CHAR = 0.25
ZIPF_BONUS_K = 2.2
LEN_BONUS_K = 0.06
UNK_PENALTY = 0.6
ALLOWED_ONE = {'в','и','к','с','а','о','у','я','I'}

In [ ]:
def word_cost(w):
    w0 = norm(w)
    L = len(w0)

    cost = BASE_PER_CHAR * L

    if w0 in WORD_COST:
        z = max(zipf_frequency(w0, 'ru'), zipf_frequency(w0, 'en'))
        cost -= ZIPF_BONUS_K * z
        cost -= LEN_BONUS_K * L
        cost = max(cost, 0.0)
    else:
        cost += UNK_PENALTY
        if L == 1 and w0 not in ALLOWED_ONE:
            cost += 3.0

    return cost

In [ ]:
SPLIT_PENALTY = 0.25
BOUNDARY_BONUS = 0.30

def dp_segment_letters(s, max_word=MAX_WORD):
    n = len(s)
    best = [1e9] * (n + 1)
    prev = [-1] * (n + 1)
    best[0] = 0.0

    for i in range(n):
        if best[i] == 1e9:
            continue
        lim = min(n, i + max_word)
        for j in range(i + 1, lim + 1):
            w = s[i:j]
            c = best[i] + word_cost(w) + SPLIT_PENALTY
            if j < n and re_boundary.search(s[j-1:j+1]):
                c -= BOUNDARY_BONUS
            if c < best[j]:
                best[j] = c
                prev[j] = i

    if prev[n] == -1:
        return [s]

    toks, cur = [], n
    while cur > 0:
        p = prev[cur]
        toks.append(s[p:cur])
        cur = p
    return toks[::-1]

In [ ]:
def split_chunks(text):
    return re.findall(rf'[{RU}{EN}{DG}]+|[^\s{RU}{EN}{DG}]+', text)

def join_with_spaces(tokens):
    out = []
    for t in tokens:
        if not out:
            out.append(t)
            continue
        if t and t[0] in PUNCT_RIGHT:
            out[-1] = out[-1].rstrip()
            out.append(t)
            continue
        if out[-1] and out[-1][-1] in PUNCT_LEFT:
            out.append(t)
            continue
        if t == '-' and out[-1] and out[-1][-1].isalnum():
            out.append(' ' + t + ' ')
            continue
        out.append(' ' + t)
    return ' '.join(' '.join(out).split())

In [ ]:
def positions_from_text(orig, spaced):
    pos = []
    i = j = 0
    while i < len(orig) and j < len(spaced):
        if spaced[j] == ' ':
            pos.append(i)
            j += 1
            continue
        if spaced[j] == orig[i]:
            i += 1
            j += 1
        else:
            j += 1
    return pos

def segment_text(text):
    chunks = split_chunks(text)
    toks = []
    for ch in chunks:
        if re.fullmatch(rf'[{RU}{EN}{DG}]+', ch):
            toks.extend(dp_segment_letters(ch))
        else:
            toks.append(ch)
    spaced = join_with_spaces(toks)
    return positions_from_text(text, spaced), spaced

In [ ]:
tests = [
    'книгавхорошемсостоянии',
    'куплюайфон14про',
    'ищудомвПодмосковье',
    'ноутбукAsusROG',
    'Былзам.министроминUSSR.',
    'Весна-выпосмотрите,каккрасиво.',
]
for t in tests:
    p, spaced = segment_text(t)
    print(t, '\n ->', p, '\n ->', spaced, '\n')


книгавхорошемсостоянии 
 -> [3, 6, 13] 
 -> кни гав хорошем состоянии 

куплюайфон14про 
 -> [5, 10, 11, 12] 
 -> куплю айфон 1 4 про 

ищудомвПодмосковье 
 -> [3, 5, 7] 
 -> ищу до мв Подмосковье 

ноутбукAsusROG 
 -> [7, 11] 
 -> ноутбук Asus ROG 

Былзам.министроминUSSR. 
 -> [3, 6, 7, 16, 18, 22] 
 -> Был зам . министром ин USSR . 

Весна-выпосмотрите,каккрасиво. 
 -> [5, 6, 8, 18, 19, 22, 29] 
 -> Весна - вы посмотрите , как красиво . 



In [ ]:
pred = []
for s in df['text_no_spaces'].astype(str):
    p, spaced = segment_text(s)
    pred.append(p)

out = pd.DataFrame({
    'id': df['id'],
    'predicted_positions': [str(x) for x in pred],
    # 'text_no_spaces': df['text_no_spaces']
})

In [ ]:
out

,id,predicted_positions
0,0,"[5, 10, 11, 12]"
1,1,"[3, 5, 7]"
2,2,"[4, 11, 13, 19, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"
...,...,...
1000,1000,"[3, 7]"
1001,1001,"[5, 6, 8, 12, 16, 19]"
1002,1002,"[5, 6, 11, 19, 24]"
1003,1003,"[5, 6, 8, 18, 19, 22, 29]"


In [ ]:
OUTPUT_CSV = '/content/submission_baseline_stat.csv'
out.to_csv(OUTPUT_CSV, index=False)


## 1 посылка
Your Mean F1 = 82.851%

такой результат у меня получился пока что. я расскажу идею. я строю скорринг алгос который для любой пары 'кандидат - слова' и решаю через дп задачку: выбрать разбиение на слова с мин сумм стоимостью. в качестве корпуса я беру готовые частоты wordfreq, чтобы не заниматься разметкой вручную. в коде я особо не писал комментов, так как планирую идею описать тут после которой код и идея станет ясной.

1) я беру топ русских слов и топ английский слов из wordfreq(через n_ru и n_en) и для каждого мне известна частота через zipf и таким образом я получаю некую стоимость. на основе значений я формирую штрафы, которые будут регулировать разделение на проблемы. но стоит отметить что я еще использую константы для подсчета штрафов, например вот


```
BASE_PER_CHAR - чем больше тем больше пробелов (антисклейка), чем меньше ну анологично соотвественно
ZIPF_BONUS_K - чем больше тем больше доверия к словарю (ну некая температура по факту) а если меньше то больше на другие штрафы полагается
LEN_BONUS_K - если больше то реже дробит длинные слова, иначе часто делит
UNK_PENALTY -  если больше то алгос будет исктаь такие разрезы чтобы слова были известны, иначе будет склеивать в длинные редкие куски
```

из этого формируется word_cost

2) best[j] это минимальная стоимость которая сформирует префикс text[:j] тогда
```
best[0] = 0
best[j] = min_i  
best[i] + word_cost(text[i:j]) + SPLIT_PENALTY
```
перебираю i ток из диапазноа [j-L, j] а L это max_word из word_cost функции

3) но в датасте есть слова где смешана и кириллица и латиница. я использую re_boundary чтобы задавать бонус к разрезу если мы видим такие переходы (ру на латиницу, либо наоборот, либо между буквой и цифрой и т.д.) вот эта часть кода
```
re_boundary = re.compile(
    rf'((?<=[{RU}])(?=[{EN}{DG}]))|'   # ру -> лат/цифры
    rf'((?<=[{EN}])(?=[{RU}{DG}]))|'   # лат -> ру/цифры
    rf'((?<=[{DG}])(?=[{RU}{EN}]))|'   # цифры -> буквы
    rf'(?<=[\p{{Ll}}])(?=\p{{Lu}})'    # aБ / aA
)
```

4) хочу так же отметить в решении как я делал постобрбаотку - я не ставил пробелы перед и после пунктуции и подобных знаков (, . ; : ! ? … ) ] } » ( [ { «) и затем получаю индексы пробелов.
```
вот мои параметры все
n_ru=200_000, n_en=150_000
BASE_PER_CHAR = 0.28
ZIPF_BONUS_K = 2.2
LEN_BONUS_K = 0.04
UNK_PENALTY = 0.8
SPLIT_PENALTY = 0.25
BOUNDARY_BONUS = 0.35
```
плюс решения что оно очень быстрое, учится на цпу и не требует никаких больших ллмок, а count based метод по факту

## 2 посылка
Your Mean F1 = 80.94%


параметры


```
WORD_COST, _MAX = build_dict(n_ru=600_000, n_en=400_000)
MAX_WORD = 32
BASE_PER_CHAR = 0.31   
ZIPF_BONUS_K  = 2.6   
LEN_BONUS_K   = 0.05  
UNK_PENALTY   = 1.1    
SPLIT_PENALTY = 0.20   
BOUNDARY_BONUS = 0.55  

```

вышло хуже

In [ ]:
tests = [
    'книгавхорошемсостоянии',
    'куплюайфон14про',
    'ищудомвПодмосковье',
    'ноутбукAsusROG',
    'Былзам.министроминUSSR.',
    'Весна-выпосмотрите,каккрасиво.',
]
for t in tests:
    p, spaced = segment_text(t)
    print(t, '\n ->', p, '\n ->', spaced, '\n')


книгавхорошемсостоянии 
 -> [3, 6, 13] 
 -> кни гав хорошем состоянии 

куплюайфон14про 
 -> [5, 10, 11, 12] 
 -> куплю айфон 1 4 про 

ищудомвПодмосковье 
 -> [2, 5, 7] 
 -> ищ удо мв Подмосковье 

ноутбукAsusROG 
 -> [7, 11] 
 -> ноутбук Asus ROG 

Былзам.министроминUSSR. 
 -> [3, 6, 7, 13, 18, 22] 
 -> Был зам . минист ромин USSR . 

Весна-выпосмотрите,каккрасиво. 
 -> [5, 6, 8, 18, 19, 22, 29] 
 -> Весна - вы посмотрите , как красиво . 



In [ ]:
pred = []
for s in df['text_no_spaces'].astype(str):
    p, spaced = segment_text(s)
    pred.append(p)

out = pd.DataFrame({
    'id': df['id'],
    'predicted_positions': [str(x) for x in pred],
    'text_no_spaces': df['text_no_spaces']
})
out

,id,predicted_positions,text_no_spaces
0,0,"[5, 10, 11, 12]",куплюайфон14про
1,1,"[2, 5, 7]",ищудомвПодмосковье
2,2,"[4, 10, 13, 19, 21]",сдаюквартирусмебельюитехникой
3,3,"[5, 10, 18]",новыйдивандоставканедорого
4,4,"[5, 10]",отдамдаромкошку
...,...,...,...
1000,1000,"[3, 7]",Янеусну.
1001,1001,"[5, 6, 8, 12, 16, 19]",Весна-яуженегреюпио.
1002,1002,"[5, 6, 11, 19, 24]",Весна-скоровырастеттрава.
1003,1003,"[5, 6, 8, 18, 19, 22, 29]","Весна-выпосмотрите,каккрасиво."


In [ ]:
OUTPUT_CSV = '/content/submission_baseline_stat2.csv'
out.to_csv(OUTPUT_CSV, index=False)


## 3 посылка
Your Mean F1 = 80.94%


параметры


```
WORD_COST, _MAX = build_dict(n_ru=600_000, n_en=400_000)
MAX_WORD = 32
BASE_PER_CHAR = 0.25
ZIPF_BONUS_K  = 2.2
LEN_BONUS_K   = 0.35  
UNK_PENALTY   = 0.6   
SPLIT_PENALTY = 0.25  
BOUNDARY_BONUS = 0.50

```

вышло хуже

In [ ]:
tests = [
    'книгавхорошемсостоянии',
    'куплюайфон14про',
    'ищудомвПодмосковье',
    'ноутбукAsusROG',
    'Былзам.министроминUSSR.',
    'Весна-выпосмотрите,каккрасиво.',
]
for t in tests:
    p, spaced = segment_text(t)
    print(t, '\n ->', p, '\n ->', spaced, '\n')


книгавхорошемсостоянии 
 -> [3, 6, 13] 
 -> кни гав хорошем состоянии 

куплюайфон14про 
 -> [5, 10, 11, 12] 
 -> куплю айфон 1 4 про 

ищудомвПодмосковье 
 -> [2, 5, 7] 
 -> ищ удо мв Подмосковье 

ноутбукAsusROG 
 -> [7, 11] 
 -> ноутбук Asus ROG 

Былзам.министроминUSSR. 
 -> [3, 6, 7, 13, 18, 22] 
 -> Был зам . минист ромин USSR . 

Весна-выпосмотрите,каккрасиво. 
 -> [5, 6, 8, 18, 19, 22, 29] 
 -> Весна - вы посмотрите , как красиво . 



In [ ]:
pred = []
for s in df['text_no_spaces'].astype(str):
    p, spaced = segment_text(s)
    pred.append(p)

out = pd.DataFrame({
    'id': df['id'],
    'predicted_positions': [str(x) for x in pred],
    'text_no_spaces': df['text_no_spaces']
})
out

,id,predicted_positions,text_no_spaces
0,0,"[5, 10, 11, 12]",куплюайфон14про
1,1,"[2, 5, 7]",ищудомвПодмосковье
2,2,"[4, 10, 13, 19, 21]",сдаюквартирусмебельюитехникой
3,3,"[5, 10, 18]",новыйдивандоставканедорого
4,4,"[5, 10]",отдамдаромкошку
...,...,...,...
1000,1000,"[3, 7]",Янеусну.
1001,1001,"[5, 6, 8, 12, 16, 19]",Весна-яуженегреюпио.
1002,1002,"[5, 6, 11, 19, 24]",Весна-скоровырастеттрава.
1003,1003,"[5, 6, 8, 18, 19, 22, 29]","Весна-выпосмотрите,каккрасиво."


In [ ]:
OUTPUT_CSV = '/content/submission_baseline_stat3.csv'
out.to_csv(OUTPUT_CSV, index=False)


# Эксперименты. seq2seq на T5
к сожалению у меня закончилось время для гпу и я не смог найти нигде гпу поэтому обучаю на цпу...

## создадим синтетик датасет

In [15]:
random.seed(42)
ru = top_n_list('ru', n=200_000, wordlist='best')
en = top_n_list('en', n=150_000, wordlist='best')
vocab = [w for w in set(ru+en) if w.isalpha() and 1 <= len(w) <= 20]
punct = [',', '.', '!', '?', ':', ';']
caps  = lambda w: w.capitalize() if random.random() < 0.15 else w
mix_token = lambda w: w if random.random() < 0.7 else w.upper() if random.random() < 0.5 else w.capitalize()

In [16]:
def make_sentence():
    n = random.randint(5, 18)
    words = []
    for _ in range(n):
        w = mix_token(random.choice(vocab))
        words.append(w)
        if random.random() < 0.10:
            words[-1] += random.choice(punct)
    if random.random() < 0.3:
        words.insert(random.randint(1, len(words)-1), str(random.randint(2, 9999)))
    if random.random() < 0.2:
        words.insert(random.randint(1, len(words)-1), random.choice(['Asus', 'ROG', 'USSR', 'iPhone', 'NVIDIA']))
    s = ' '.join(words)
    s = caps(s.split()[0]) + ' ' + ' '.join(s.split()[1:]) if len(s.split()) > 1 else s
    return s

In [17]:
def make_pairs(k=50_000):
    data = []
    for _ in range(k):
        with_space = make_sentence()
        no_space = with_space.replace(' ', '')
        data.append({'input': no_space, 'target': with_space})
    return data

In [18]:
train = make_pairs(45_000)
valid = make_pairs(2_000)
test  = make_pairs(2_000)

ds = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(train)),
    'validation': Dataset.from_pandas(pd.DataFrame(valid)),
    'test': Dataset.from_pandas(pd.DataFrame(test))
})
ds

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 45000
    })
    validation: Dataset({
        features: ['input', 'target'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 2000
    })
})

## токенизация и модель


In [19]:
print(torch.__version__)

2.8.0+cu126


In [20]:
model_name = 'google/byt5-small'
tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(384, 1472)
  (encoder): T5Stack(
    (embed_tokens): Embedding(384, 1472)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1472, out_features=384, bias=False)
              (k): Linear(in_features=1472, out_features=384, bias=False)
              (v): Linear(in_features=1472, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=1472, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1472, out_features=3584, bias=False)
              (wi_1): Linear(in_features=1472, out_features=3584, bias=False)
              (w

In [21]:
max_len_in  = 256
max_len_out = 300

def preprocess(batch):
    x = tok(batch['input'], max_length=max_len_in, truncation=True)
    y = tok(text_target=batch['target'], max_length=max_len_out, truncation=True)
    x['labels'] = y['input_ids']
    return x

ds_tok = ds.map(preprocess, batched=True, remove_columns=ds['train'].column_names)
ds_tok

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 45000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [22]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer
data_collator = DataCollatorForSeq2Seq(tok, model=model)

args = TrainingArguments(
    output_dir="//Applications/programming/vscode_projects/avito_ds/byt5_spaces",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    learning_rate=3e-4,
    fp16=torch.cuda.is_available(),
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["validation"],
    data_collator=data_collator,
    tokenizer=tok,
)
trainer.train()

/tmp/ipython-input-919829095.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
def restore_spaces(text, max_new_tokens=350):
    enc = tok(text, return_tensors="pt")
    with torch.no_grad():
        out = model.generate(
            **enc, max_new_tokens=max_new_tokens, num_beams=4, length_penalty=0.2
        )
    return tok.decode(out[0], skip_special_tokens=True)


def positions_from_text(orig, spaced):
    pos = []
    i = j = 0
    while i < len(orig) and j < len(spaced):
        if spaced[j] == " ":
            pos.append(i)
            j += 1
            continue
        if spaced[j] == orig[i]:
            i += 1
            j += 1
        else:
            j += 1
    return pos

In [ ]:
tests = [
    "книгавхорошемсостоянии",
    "куплюайфон14про",
    "ищудомвПодмосковье",
    "ноутбукAsusROG",
    "Былзам.министроминUSSR.",
    "Весна-выпосмотрите,каккрасиво.",
]
for t in tests:
    spaced = restore_spaces(t)
    pos = positions_from_text(t, spaced)
    print(t, "\n->", spaced, "\n->", pos, "\n")

In [ ]:
pred = []
for s in df["text_no_spaces"].astype(str):
    p, spaced = segment_text(s)
    pred.append(p)

out = pd.DataFrame(
    {
        "id": df["id"],
        "predicted_positions": [str(x) for x in pred],
        "text_no_spaces": df["text_no_spaces"],
    }
)
out

In [ ]:
OUTPUT_CSV = "/home/jupyter/datasphere/project/submission_baseline_stat4.csv"
out.to_csv(OUTPUT_CSV, index=False)